Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Distributed Image Classification with PyTorch

In this tutorial, you will train a PyTorch model on the [CIFAR10](http://www.cs.toronto.edu/~kriz/cifar.html) dataset using distributed training with PyTorch's `DistributedDataParallel` module across a Azure Stack Hub CPU Kubernetes cluster. The training dataset are stored on Azure Machine Learning datastore backed up by Azure Stack Hub Storage account.

## Prerequisites

*    [ A Kubernetes cluster deployed on Azure Stack Hub, connected to Azure through ARC](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/AML-ARC-Compute.md).
     

*     [Datastore setup in Azure Machine Learning workspace backed up by Azure Stack Hub storage account](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/Train-AzureArc.md) 


*      Last but not least, you need to be able to run a Notebook. 

   If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the configuration Notebook located at [here](https://github.com/Azure/MachineLearningNotebooks) first. This sets you up with a working config file that has information on your workspace, subscription id, etc.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize AzureML workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`. 

If you haven't done already please go to `config.json` file and fill in your workspace information.

In [ ]:
from azureml.core.workspace import Workspace,  ComputeTarget
from azureml.exceptions import ComputeTargetException

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Prepare the dataset

You may download cifar10 dataset from [cifar10-data](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz). Create folder "cifar10-data" under working directory of this notebook, then  copy "cifar-10-python.tar.gz" to folder "cifar10-data". The following cell will upload "cifar-10-python.tar.gz" to datastore of the workspace, and finally registered as dataset in the workspace. 

Upload and dataset registration take about 3 mins.

In [ ]:
import os
from azureml.core import Datastore, Dataset

dataset_name = 'cifar10_olando'
datastore_name = "ashstoreolando"

if dataset_name not  in ws.datasets:
    datastore =  Datastore.get(ws, datastore_name)
    
    src_dir, target_path = 'cifar10-data', 'cifar10-data-ash' #assuming cifar-10-python.tar.gz is in folder cifar10-data
    
    # upload data from local to AML datastore:
    datastore.upload(src_dir, target_path)

    # register data uploaded as AML dataset:
    datastore_paths = [(datastore, target_path)]
    cifar_ds = Dataset.File.from_files(path=datastore_paths)
    cifar_ds.register(ws, dataset_name, "CIFAR-10 images from https://www.cs.toronto.edu/~kriz/cifar.html")
        
dataset_ash = ws.datasets[dataset_name]

## Setup compute target

Find the attach name for the Arc enabled  Azure Stack Hub kubernetes cluster in your AzureML workspace to create a ComputeTarget:

In [ ]:
from azureml.contrib.core.compute.arckubernetescompute import ArcKubernetesCompute

attach_name = "sl-d2-o-arc"
arcK_target = ArcKubernetesCompute(ws, attach_name)

## Configure the training job and submit

### Create an experiement

In [ ]:
from azureml.core import Experiment

experiment_name = 'pytorch-cifar-distr'
experiment = Experiment(ws, name=experiment_name)

### Create an environment

In [ ]:
from azureml.core import Environment

pytorch_env = Environment.from_conda_specification(name = 'pytorch-1.6-cpu', file_path = 'pytorch-script/conda_dependencies.yml')

# Specify a CPU base image
pytorch_env.docker.enabled = True
pytorch_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04'


### Configure the training job: torch.distributed with GLOO backend

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on.

In order to run a distributed PyTorch job with **torch.distributed** using the GLOO backend, create a `PyTorchConfiguration` and pass it to the `distributed_job_config` parameter of the ScriptRunConfig constructor. Specify `communication_backend='Gloo'` in the PyTorchConfiguration. The below code will configure node_count = 2. These is the number of worker nodes. The number of  distributed jobs will be 3 if one master node is used.  GLOO backend which is recommended backend for communications between CPUs.

Tthe script for distributed training of CIFAR10 is already provided for you at `pytorch-script/cifar_dist_main.py`. In practice, you should be able to take any custom PyTorch training script as is and run it with Azure ML without having to modify your code.

With node_count=2, training for one epoch may take 20 mins with vm size comparable to Standard_DS3_v2

In [ ]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
from azureml.core import Dataset
import os

dataset_ash = Dataset.get_by_name(ws, name=dataset_name)
args = [
        '--data-folder', dataset_ash.as_mount(),
        '--dist-backend', 'gloo',
       '--epochs', 1 #20
           ]

distributed_job_config=PyTorchConfiguration(communication_backend='Gloo', node_count=2) #configuring AML pytorch config

project_folder = "pytorch-script"
run_script = "cifar_dist_main.py"
src = ScriptRunConfig(
                     source_directory=project_folder,
                      script=run_script,
                      arguments=args,
                      compute_target=arcK_target,
                      environment=pytorch_env,
                      distributed_job_config=distributed_job_config)

### Submit the job
Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.

In [ ]:
run = experiment.submit(src)
run.wait_for_completion(show_output=True) # this provides a verbose log

### Register the model

Register the trained model.

In [ ]:
register_model_name = 'cifar10torch'
model = run.register_model(model_name=register_model_name, model_path='outputs/cifar10torch.pt')

The machine learning model named "cifar10torch" should be registered in your AzureML workspace.

## Test the registered model

To test the trained model, you can create (or use existing) a AKS cluster for serving the model using AML deployment.

Note: You will test the model in azure cloud because AzureML inferencing on Azure Stack Hub in currently unavailable.

In [ ]:
from azureml.core import Environment, Workspace, Model, ComputeTarget
from azureml.core.compute import AksCompute
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.compute_target import ComputeTargetException
import numpy as np
import json

### Provision the AKS cluster

This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it. It may take 5 mins to create a new AKS cluster.

In [ ]:
ws = Workspace.from_config()

# Choose a name for your AKS cluster
aks_name = 'aks-service-2'

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    is_new_compute  = False
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)
    is_new_compute  = True
    
print("using compute target: ", aks_target.name)

### Deploy the model

In [ ]:
env = Environment.from_conda_specification(name='pytorch-cifar', file_path='pytorch-script/conda_dependencies.yml')

inference_config = InferenceConfig(entry_script='score_pytorch.py', environment=env)
deploy_config = AksWebservice.deploy_configuration()

model = ws.models[register_model_name]
service_name = 'cifartorchservice4'

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deploy_config,
                       deployment_target=aks_target,
                       overwrite=True)

service.wait_for_deployment(show_output=True)

### Test with inputs

For testing purpose, first image (a cat) from test batch is extracted (it is saved at test_imgs/test_img_0_cat.jpg). It is shown here: 

![fishy](test_imgs/test_img_0_cat.jpg)

After some data process, the image converted to json as input for the trained model. The outputs are logits for each class per image:

In [ ]:
with open("cifar_test_input_pytorch.json", "r") as fp:
    inputs_json = json.load(fp)
    
inputs = json.dumps(inputs_json)
resp = service.run(inputs)
predicts = resp["predicts"]
print(predicts)

Then you can easily get the predictions of labels:

In [ ]:
import numpy as np
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')
np_predicts = np.array(predicts)
pred_indexes = np.argmax(np_predicts, 1)

predict_labels = [classes[i] for i in pred_indexes]
print(predict_labels)

Depending on your model, you may or may not get the correct label which is "cat".

### Delete the newly created cluster

Note: This is important if you wish to avoid the cost of this cluster

In [ ]:
if is_new_compute:
    aks_target.delete()
    service.delete()

## Next steps

1. Learn how to [download model then upload to Azure Storage blobs](../AML-model-download-upload.ipynb)
2. Learn how to [inference using KFServing with model in Azure Storage Blobs](https://aka.ms/kfas)
3. Learn Pipeline Steps with [Object Segmentation](../object-segmentation-on-azure-stack/)